In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

query1 = '''
SELECT
  *
FROM
  houseprices
'''

df = pd.read_sql_query(query1, con=engine)
engine.dispose()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

In [4]:
df['total_square_feet'] = df['totalbsmtsf'] + df['grlivarea'] 

In [5]:
features = ['exterqual', 'kitchenqual', 'has_central_air', 'neighborhood', 'total_square_feet', 'yearbuilt', 'saleprice']

In [6]:
df['has_central_air'] = pd.get_dummies(df['centralair'], drop_first=True)

In [7]:
df_features = df[features]

In [8]:
df_features = pd.concat([df_features, pd.get_dummies(df_features['kitchenqual']
                                                     , prefix='kitchen_quality_', drop_first=True)], axis=1)

In [9]:
df_features = pd.concat([df_features, pd.get_dummies(df_features['neighborhood']
                                                     , prefix='neighborhood_', drop_first=True)], axis=1)

In [10]:
df_features = pd.concat([df_features, pd.get_dummies(df_features['exterqual']
                                                    , prefix='external_quality', drop_first=True)], axis=1)

In [11]:
df_features.drop(['exterqual', 'kitchenqual', 'neighborhood'], inplace=True, axis=1)

In [12]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   has_central_air        1460 non-null   uint8
 1   total_square_feet      1460 non-null   int64
 2   yearbuilt              1460 non-null   int64
 3   saleprice              1460 non-null   int64
 4   kitchen_quality__Fa    1460 non-null   uint8
 5   kitchen_quality__Gd    1460 non-null   uint8
 6   kitchen_quality__TA    1460 non-null   uint8
 7   neighborhood__Blueste  1460 non-null   uint8
 8   neighborhood__BrDale   1460 non-null   uint8
 9   neighborhood__BrkSide  1460 non-null   uint8
 10  neighborhood__ClearCr  1460 non-null   uint8
 11  neighborhood__CollgCr  1460 non-null   uint8
 12  neighborhood__Crawfor  1460 non-null   uint8
 13  neighborhood__Edwards  1460 non-null   uint8
 14  neighborhood__Gilbert  1460 non-null   uint8
 15  neighborhood__IDOTRR   1460 non-null  

In [13]:
for column in ['saleprice', 'yearbuilt', 'total_square_feet']:
    df_features[column] = winsorize(df_features[column], (.05, .05))

In [14]:
X = df_features.drop('saleprice', axis=1)
y = df_features['saleprice']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

alphas = [np.power(10.0,p) for p in np.arange(-10,40,1)]

In [17]:
lrm = LinearRegression()

lrm.fit(X_train, y_train)

y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in training set is: {}".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

R-squared of the model in training set is: 0.8569960809627851
-----Test set statistics-----
R-squared of the model in test set is: 0.8417840966212042
Mean absolute error of the prediction is: 18558.83108091505
Mean squared error of the prediction is: 615169259.626791
Root mean squared error of the prediction is: 24802.605903952735
Mean absolute percentage error of the prediction is: 11.405752115558375


In [18]:
lasso_cv = LassoCV(alphas=alphas, cv=5)

lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print("Best alpha value is: {}".format(lasso_cv.alpha_))
print("R-squared of the model in training set is: {}".format(lasso_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 1.0
R-squared of the model in training set is: 0.856984741718274
-----Test set statistics-----
R-squared of the model in test set is: 0.8421665998665651
Mean absolute error of the prediction is: 18546.192944241706
Mean squared error of the prediction is: 613682024.5686938
Root mean squared error of the prediction is: 24772.60633378518
Mean absolute percentage error of the prediction is: 11.402614004676233


In [19]:
ridge_cv = RidgeCV(alphas=alphas, cv=5)

ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


Best alpha value is: 0.1
R-squared of the model in training set is: 0.8569525779227938
-----Test set statistics-----
R-squared of the model in test set is: 0.8425422173881973
Mean absolute error of the prediction is: 18531.874181423296
Mean squared error of the prediction is: 612221562.328484
Root mean squared error of the prediction is: 24743.111411633017
Mean absolute percentage error of the prediction is: 11.399309625057974


In [20]:

elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5)

elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print("Best alpha value is: {}".format(elasticnet_cv.alpha_))
print("R-squared of the model in training set is: {}".format(elasticnet_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(elasticnet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.0001
R-squared of the model in training set is: 0.8569786149658353
-----Test set statistics-----
R-squared of the model in test set is: 0.8422884896042362
Mean absolute error of the prediction is: 18541.581330114914
Mean squared error of the prediction is: 613208097.371251
Root mean squared error of the prediction is: 24763.03893651284
Mean absolute percentage error of the prediction is: 11.401617268374515


For this model Ridge regularization is best. However this model is still not ready for deployment. There are underlying issues with the features and this exercise is only to demonstrate the differences that regularaization makes. I wonder if a better selection of features would lead to more pronounced differences between outcomes.

I also wonder about adding the constant to these models I'm unclear how that is done for these other methods. Can I add the constant with statsmodel the same way I would for OLS then apply other methods of regression?